## **Installing packages**

In [ ]:
!pip3 install torch torchvision torchaudio
!pip install 'tqdm'

In [ ]:
!pip install tensorboardX

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.0 MB/s eta 0:00:00


## **Libraries**

In [ ]:
import os
import os.path
import sys
import logging

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader, Dataset
from torch.backends import cudnn
import torch.cuda.amp as amp
from torch.autograd import Function
import torch.nn.functional as F

import torchvision
from torchvision.transforms import transforms
#from torchvision.models import alexnet

from PIL import Image
from tqdm import tqdm
import numpy as np
from tensorboardX import SummaryWriter

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import random

from pathlib import Path
import json
import random


logger = logging.getLogger()

## **Import**

### Importing datasets

In [ ]:
from google.colab import drive

drive.mount('/content/Drive')

cityscapes = True
gta5 = True

if not os.path.isdir(f'/content/Cityscapes') and cityscapes:
  !jar xvf  "/content/Drive/MyDrive/Colab Notebooks/AML/Cityscapes.zip"
if not os.path.isdir(f'/content/GTA5') and gta5:
  !jar xvf  "/content/Drive/MyDrive/Colab Notebooks/AML/GTA5.zip"

Output streaming troncato alle ultime 5000 righe.
 inflated: GTA5/images/00002.png
 inflated: GTA5/images/00003.png
 inflated: GTA5/images/00004.png
 inflated: GTA5/images/00005.png
 inflated: GTA5/images/00006.png
 inflated: GTA5/images/00007.png
 inflated: GTA5/images/00008.png
 inflated: GTA5/images/00009.png
 inflated: GTA5/images/00010.png
 inflated: GTA5/images/00011.png
 inflated: GTA5/images/00012.png
 inflated: GTA5/images/00013.png
 inflated: GTA5/images/00014.png
 inflated: GTA5/images/00015.png
 inflated: GTA5/images/00016.png
 inflated: GTA5/images/00017.png
 inflated: GTA5/images/00018.png
 inflated: GTA5/images/00019.png
 inflated: GTA5/images/00020.png
 inflated: GTA5/images/00021.png
 inflated: GTA5/images/00022.png
 inflated: GTA5/images/00023.png
 inflated: GTA5/images/00024.png
 inflated: GTA5/images/00025.png
 inflated: GTA5/images/00026.png
 inflated: GTA5/images/00027.png
 inflated: GTA5/images/00028.png
 inflated: GTA5/images/00029.png
 inflated: GTA5/images/000

### Import methods from TA's repository

In [ ]:
# cloning github repo for model (BiseNet with STDC) and utils, I rewrote manually the Train.py and Cityscapes.py below
import pathlib
print(pathlib.Path.cwd())
!git clone https://github.com/ClaudiaCuttano/AML_Semantic_DA.git


# importing stuff from the repo we just cloned

# copied from train.py
from AML_Semantic_DA.model.model_stages import BiSeNet  # see https://github.com/ClaudiaCuttano/AML_Semantic_DA/blob/master/model/model_stages.py
from AML_Semantic_DA.utils import poly_lr_scheduler, reverse_one_hot, compute_global_accuracy, fast_hist, per_class_iu # see https://github.com/ClaudiaCuttano/AML_Semantic_DA/blob/master/utils.py

/content
Cloning into 'AML_Semantic_DA'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 18 (delta 0), reused 0 (delta 0), pack-reused 11
Receiving objects: 100% (18/18), 10.88 KiB | 10.88 MiB/s, done.


In [ ]:
# taken from the original train.py, not sure what it's for
def str2bool(v):
    """
    It converts a string into a boolean
    """

    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise 'Unsupported value encountered.'

### Import methods for GTA5
Clone repo useful to give a label for each pixel.

In [ ]:
import pathlib
print(pathlib.Path.cwd())
!git clone https://github.com/MichaelFan01/STDC-Seg.git STDC_seg
# importing stuff from the repo we just cloned

/content
Cloning into 'STDC_seg'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 98 (delta 18), reused 8 (delta 8), pack-reused 70
Receiving objects: 100% (98/98), 1.93 MiB | 5.85 MiB/s, done.
Resolving deltas: 100% (24/24), done.


## **Data loader classes + Preprocessing**

### Data Pre Processing
We need to do pre-processing only on **training set** (and not on validation set because on the last one we work with 1024 x 2048 pictures)

In [ ]:
#transforms.CenterCrop((512,1024))
train_transform = transforms.Compose([transforms.Resize((512,1024), transforms.InterpolationMode.BILINEAR),
                                      transforms.ToTensor()
                                      ])
label_transform = transforms.Resize((512,1024), transforms.InterpolationMode.NEAREST)
eval_transform = transforms.Compose([transforms.ToTensor()])


gta_train_transform = transforms.Compose([transforms.Resize((512,1024), transforms.InterpolationMode.BILINEAR),
                                          transforms.ToTensor()
                                          ])
gta_label_transform = transforms.Resize((512,1024), transforms.InterpolationMode.NEAREST)
gta_val_transform = transforms.Compose([transforms.ToTensor()])


#data augmentation
bright_t = transforms.ColorJitter(brightness=[1,2])
contrast_t = transforms.ColorJitter(contrast = [2,5])
saturation_t = transforms.ColorJitter(saturation = [1,3])
hue_t = transforms.ColorJitter(hue = 0.2)
gs_t = transforms.Grayscale(3)
hflip_t = transforms.RandomHorizontalFlip(p = 1)
cc_t = transforms.CenterCrop((256,512))

augmentation_transforms = [bright_t, contrast_t, saturation_t, hue_t, gs_t, hflip_t, cc_t]

#rp_t = transforms.RandomPerspective(p = 1, distortion_scale = 0.5)
#augmentation_transforms = [bright_t, contrast_t, saturation_t, hue_t, gs_t, hflip_t, rp_t]

### Cityscapes

In [ ]:
class CityScapes(Dataset):
    def __init__(self, base_root, mode):
        super(CityScapes, self).__init__()

        self.mode = mode
        self.image_paths = [] # images
        self.mask_paths_colored = [] # colored images
        self.mask_paths_bw = [] # labels

        assert(mode == 'train' or mode == 'val')  # just checking for potential issues
        image_folder = f'{base_root}images/{mode}'

        for root, dirs, files in os.walk(image_folder):
            for file_name in files:
                image_path = f'{root}/{file_name}'
                assert(Path(image_path).is_file())
                self.image_paths.append(image_path)

                mask_path_bw = image_path.replace('leftImg8bit', 'gtFine_labelTrainIds')
                mask_path_bw = mask_path_bw.replace('/images/', '/gtFine/')
                assert(Path(mask_path_bw).is_file())
                self.mask_paths_bw.append(mask_path_bw)

                #mask_path_colored = image_path.replace('leftImg8bit', 'gtFine_labelTrainIds')
                #mask_path_colored = mask_path_colored.replace('/images/', '/gtFine/')
                #assert(Path(mask_path_colored).is_file())
                #self.mask_paths_colored.append(mask_path_colored)

        if self.mode == 'train':
            self.image_transform = train_transform
        if self.mode == 'val':
            self.image_transform = eval_transform
        self.label_transform = label_transform

        assert (len(self.image_paths) != 0)
        assert (len(self.image_paths) == len(self.mask_paths_bw))
        #assert (len(self.image_paths) == len(self.mask_paths_colored))


    def __getitem__(self, index):

        image = Image.open(self.image_paths[index]).convert('RGB')
        label = Image.open(self.mask_paths_bw[index])

        if self.mode == 'train':
            label = np.array(self.label_transform(label))[np.newaxis, :]
        else:
            label = np.array(label)[np.newaxis, :]

        image = self.image_transform(image)

        return image, label


    def __len__(self):
        return len(self.image_paths)

### GTA5

In [ ]:
class GTA5(Dataset):
    def __init__(self, base_root, mode, augmentation=False, train_test_rateo=2/3):
        super(GTA5, self).__init__()

        self.mode = mode
        self.image_paths = [] # images
        self.label_paths = [] # labels
        self.train_test_rateo = train_test_rateo
        self.augmentation = augmentation
        with open('STDC_seg/cityscapes_info.json', 'r') as fr:
            labels_info = json.load(fr)
        self.label_map = {el['id']: el['trainId'] for el in labels_info}
        self.label_map.update({34 : 255})

        assert(mode == 'train' or mode == 'val')  # just checking for potential issues

        image_folder = f'{base_root}images'

        for root, dirs, files in os.walk(image_folder):
            for file_name in files:
                image_path = f'{root}/{file_name}'
                assert(Path(image_path).is_file())
                self.image_paths.append(image_path)

                label_path = image_path.replace('/images/', '/labels/')
                assert(Path(label_path).is_file())
                self.label_paths.append(label_path)

        l = int(len(self.image_paths) * self.train_test_rateo)
        if self.mode == 'train':
            self.image_paths = self.image_paths[:l]
            self.label_paths = self.label_paths[:l]
            self.image_transform = gta_train_transform
            self.label_transform = gta_label_transform
        elif self.mode == 'val':
            self.image_paths = self.image_paths[l:]
            self.label_paths = self.label_paths[l:]
            self.image_transform = gta_val_transform

        assert (len(self.image_paths) != 0)
        assert (len(self.image_paths) == len(self.label_paths))


    def __getitem__(self, index):

        image = Image.open(self.image_paths[index]).convert('RGB')
        image = self.image_transform(image)

        label = Image.open(self.label_paths[index])
        if self.mode == 'train':
            label = self.label_transform(label)

        if self.augmentation and random.choice([True, False]) and self.mode == 'train':
            idx = random.randint(0, 6)

            image = augmentation_transforms[idx](image)

            if hflip_t is augmentation_transforms[idx] or cc_t is augmentation_transforms[idx]:
                label = augmentation_transforms[idx](label)

            if cc_t is augmentation_transforms[idx]:
                rimage_t = transforms.Resize((512,1024), transforms.InterpolationMode.BILINEAR, antialias=None)
                rlabel_t = transforms.Resize((512,1024), transforms.InterpolationMode.NEAREST, antialias=None)
                image = rimage_t(image)
                label = augmentation_transforms[idx](label)
                label = rlabel_t(label)

        label = np.array(label).astype(np.int64)[np.newaxis, :]
        label = self.convert_labels(label)

        return image, label


    def convert_labels(self, label):
        for k, v in self.label_map.items():
            label[label == k] = v
        return label


    def __len__(self):
        return len(self.image_paths)


## **Architecture**
BiSeNet as Domain Adaptation Neural Network (DANN)

In [ ]:
class FCDiscriminator(nn.Module):

    def __init__(self, num_classes, ndf = 64):
        super(FCDiscriminator, self).__init__()

        self.conv1 = nn.Conv2d(num_classes, ndf, kernel_size=4, stride=2, padding=1)
        self.conv2 = nn.Conv2d(ndf, ndf*2, kernel_size=4, stride=2, padding=1)
        self.conv3 = nn.Conv2d(ndf*2, ndf*4, kernel_size=4, stride=2, padding=1)
        self.conv4 = nn.Conv2d(ndf*4, ndf*8, kernel_size=4, stride=2, padding=1)
        self.classifier = nn.Conv2d(ndf*8, 1, kernel_size=4, stride=2, padding=1)

        self.leaky_relu = nn.LeakyReLU(negative_slope=0.2, inplace=True)
        self.up_sample = nn.Upsample(scale_factor=32, mode='bilinear')
        #self.sigmoid = nn.Sigmoid()


    def forward(self, x):
        x = self.conv1(x)
        x = self.leaky_relu(x)
        x = self.conv2(x)
        x = self.leaky_relu(x)
        x = self.conv3(x)
        x = self.leaky_relu(x)
        x = self.conv4(x)
        x = self.leaky_relu(x)
        x = self.classifier(x)
        x = self.up_sample(x)
        #x = self.sigmoid(x)

        return x

## **Train + Validation**

### Base Training

In [ ]:
from torch.autograd import Variable

def train(args, model, optimizer, dataloader_train, dataloader_val):
    print("start train without domain adaptation")
    # for SummaryWriter read (https://pytorch.org/docs/stable/tensorboard.html)
    writer = SummaryWriter(log_dir='/content/Drive/MyDrive/AML project/logs', comment=''.format(args.optimizer)) # log is in run/ folder

    scaler = amp.GradScaler()

    loss_func = torch.nn.CrossEntropyLoss(ignore_index=255)
    max_miou = 0
    step = 0

    for epoch in range(args.epoch_start_i+1, args.num_epochs+1):
        lr = poly_lr_scheduler(optimizer, args.learning_rate, iter=epoch, max_iter=args.num_epochs)
        model.train() # Sets module in training mode
        tq = tqdm(total=len(dataloader_train) * args.batch_size)
        tq.set_description('epoch %d, lr %f' % (epoch, lr))
        loss_record = []

        for i, (data, label) in enumerate(dataloader_train):
            data = data.cuda()
            label = label.long().cuda()
            optimizer.zero_grad() # Zero-ing the gradients
            # From: [batch_size, height, width, channels]
            # To: [batch_size, channels, height, width]
            #data = data.permute(0, 3, 1, 2)

            with amp.autocast():
                output, out16, out32 = model(data)  # Forward pass to the network
                # Compute loss based on output and ground truth
                loss1 = loss_func(output, label.squeeze(1))
                loss2 = loss_func(out16, label.squeeze(1))
                loss3 = loss_func(out32, label.squeeze(1))
                loss = loss1 + loss2 + loss3  # sum of losses

            # Compute gradients for each layer and update weights
            scaler.scale(loss).backward() # backward pass: computes gradients
            scaler.step(optimizer)        # update weights based on accumulated gradients
            scaler.update()

            tq.update(args.batch_size)
            tq.set_postfix(loss='%.6f' % loss)
            step += 1
            writer.add_scalar('loss_step', loss, step)
            loss_record.append(loss.item())
        tq.close()
        loss_train_mean = np.mean(loss_record)
        writer.add_scalar('epoch/loss_epoch_train', float(loss_train_mean), epoch)
        print('loss for train : %f' % (loss_train_mean))

        if epoch % args.checkpoint_step == 0 and epoch != 0:
            import os
            if not os.path.isdir(args.save_model_path):
                os.mkdir(args.save_model_path)
            torch.save(model.module.state_dict(), f'{args.save_model_path}Saved_model_epoch_{epoch}.pth')

        if epoch % args.validation_step == 0 and epoch != args.num_epochs:
            precision, miou = val(args, model, dataloader_val)  # val() function call
            if miou > max_miou:
                max_miou = miou
                import os
                os.makedirs(args.save_model_path, exist_ok=True)
                torch.save(model.module.state_dict(), f'{args.save_model_path}Best_model_epoch_{epoch}.pth')
            writer.add_scalar('epoch/precision_val', precision, epoch)
            writer.add_scalar('epoch/miou val', miou, epoch)

### Domain Adaptation Training

In [ ]:
from torch.autograd import Variable


def train_da(args, model, optimizer, model_D, optimizer_D, dataloader_train, dataloader_val, domain_adapt=False, dataloader_target=None):
    print("start train with domain adaptation")
    # for SummaryWriter read (https://pytorch.org/docs/stable/tensorboard.html)
    writer = SummaryWriter(log_dir='/content/Drive/MyDrive/AML project/logs', comment=''.format(args.optimizer)) # log is in run/ folder

    scaler = amp.GradScaler()

    loss_func_G = torch.nn.CrossEntropyLoss(ignore_index=255)
    loss_func_adv = torch.nn.BCEWithLogitsLoss()
    loss_func_D = torch.nn.BCEWithLogitsLoss()

    max_miou = 0
    step = 0

    # see (https://www.github.com/wasidennis/AdaptSegNet/blob/master/train_gta2cityscapes_multi)
    print("Train DA")

    LAMBDA_ADV_TARGET = 0.001

    dataloader_len = min(len(dataloader_train), len(dataloader_target))
    for epoch in range(args.epoch_start_i+1, args.num_epochs+1):
        lr = poly_lr_scheduler(optimizer, args.learning_rate, iter=epoch-1, max_iter=args.num_epochs)
        lr_D = poly_lr_scheduler(optimizer_D, args.learning_rate_D, iter=epoch-1, max_iter=args.num_epochs)

        model.train()
        model_D.train()

        tq = tqdm(total=dataloader_len * args.batch_size)

        tq.set_description('epoch %d, lr %f, lr_discriminator %f' % (epoch, lr, lr_D))

        # set the ground truth for the discriminator
        source_label = 0
        target_label = 1
        # initiate lists to track the losses
        loss_G_record = []                                                       # track the Segmentation loss
        loss_adv_record = []                                                     # track the advarsirial loss
        loss_D_record = []                                                       # track the discriminator loss

        source_train_loader = enumerate(dataloader_train)
        s_size = len(dataloader_train)
        target_loader = enumerate(dataloader_target)
        t_size = len(dataloader_target)

        for i in range(dataloader_len):

            optimizer.zero_grad()
            optimizer_D.zero_grad()

            # =====================================
            # train Generator G:
            # =====================================

            for param in model_D.parameters():
                param.requires_grad = False

            # Train with source:
            # =================================

            _, batch = next(source_train_loader)
            data, label = batch
            data = data.cuda()
            label = label.long().cuda()

            with amp.autocast():
                output_s, out16, out32 = model(data)
                loss1 = loss_func_G(output_s, label.squeeze(1))
                loss2 = loss_func_G(out16, label.squeeze(1))
                loss3 = loss_func_G(out32, label.squeeze(1))
                loss_G = loss1 + loss2 + loss3

            scaler.scale(loss_G).backward()

            # Train with target:
            # =================================

            _, batch = next(target_loader)

            data, _ = batch
            data = data.cuda()
            with amp.autocast():

                output_t, _, _ = model(data)
                D_out = model_D(F.softmax(output_t))
                loss_adv = loss_func_adv(D_out , Variable(torch.FloatTensor(D_out.data.size()).fill_(source_label)).cuda() )  # Generator try to fool the discriminator
                loss_adv = loss_adv * LAMBDA_ADV_TARGET

            scaler.scale(loss_adv).backward()

            # =====================================
            # train Discriminator D:
            # =====================================

            for param in model_D.parameters():
                param.requires_grad = True

            # Train with source:
            # =================================

            output_s = output_s.detach()
            with amp.autocast():
                D_out = model_D(F.softmax(output_s))                                                                   # we feed the discriminator with the output of the G-model
                loss_D = loss_func_D(D_out, Variable(torch.FloatTensor(D_out.data.size()).fill_(source_label)).cuda())
                loss_D = loss_D / 2
            scaler.scale(loss_D).backward()

            # Train with target:
            # =================================

            output_t = output_t.detach()
            with amp.autocast():
                D_out = model_D(F.softmax(output_t))  # we feed the discriminator with the output of the model
                loss_D = loss_func_D(D_out, Variable(torch.FloatTensor(D_out.data.size()).fill_(target_label)).cuda())  # add the adversarial loss
                loss_D = loss_D / 2
            scaler.scale(loss_D).backward()

            tq.update(args.batch_size)
            losses = {"loss_seg" : '%.6f' %(loss_G.item())  , "loss_adv" : '%.6f' %(loss_adv.item()) , "loss_D" : '%.6f'%(loss_D.item()) } # add dictionary to print losses
            tq.set_postfix(losses)

            loss_G_record.append(loss_G.item())
            loss_adv_record.append(loss_adv.item())
            loss_D_record.append(loss_D.item())
            step += 1
            writer.add_scalar('loss_G_step', loss_G, step)  # track the segmentation loss
            writer.add_scalar('loss_adv_step', loss_adv, step)  # track the adversarial loss
            writer.add_scalar('loss_D_step', loss_D, step)  # track the discreminator loss
            scaler.step(optimizer)  # update the optimizer for genarator
            scaler.step(optimizer_D)  # update the optimizer for discriminator
            scaler.update()
        tq.close()

        loss_seg_record_mean = np.mean(loss_G_record)
        loss_adv_record_mean = np.mean(loss_adv_record)
        loss_D_record_mean = np.mean(loss_D_record)
        loss_mean = np.mean([loss_seg_record_mean, loss_adv_record_mean, loss_D_record_mean])
        writer.add_scalar('epoch/loss_epoch_train', float(loss_mean), epoch)
        print(f'loss for train :\n - Segmentation: {loss_seg_record_mean}\n - Adversarial: {loss_adv_record_mean}\n - Discriminator: {loss_D_record_mean}')

        if epoch % args.checkpoint_step == 0 and epoch != 0:
            import os
            if not os.path.isdir(args.save_model_path):
                os.mkdir(args.save_model_path)
            torch.save(model.module.state_dict(), f'{args.save_model_path}DA_Saved_model_epoch_{epoch}.pth')

        if epoch % args.validation_step == 0 and epoch != args.num_epochs:
            precision, miou = val(args, model, dataloader_val)  # val() function call
            if miou > max_miou:
                max_miou = miou
                import os
                os.makedirs(args.save_model_path, exist_ok=True)
                torch.save(model.module.state_dict(), f'{args.save_model_path}DA_Best_model_epoch_{epoch}.pth')
            writer.add_scalar('epoch/precision_val', precision, epoch)
            writer.add_scalar('epoch/miou val', miou, epoch)

### Validation

In [ ]:
def val(args, model, dataloader):
    print('start val!')
    with torch.no_grad():
        model.eval()
        precision_record = []
        hist = np.zeros((args.num_classes, args.num_classes))
        tq = tqdm(total=len(dataloader))
        tq.set_description('Validation')
        for i, (data, label) in enumerate(dataloader):
            label = label.type(torch.LongTensor)
            data = data.cuda()
            label = label.long().cuda()

            # get RGB predict image
            predict, _, _ = model(data)
            predict = predict.squeeze(0)
            predict = reverse_one_hot(predict)
            predict = np.array(predict.cpu())

            # get RGB label image
            label = label.squeeze()
            label = np.array(label.cpu())

            # compute per pixel accuracy
            precision = compute_global_accuracy(predict, label)
            hist += fast_hist(label.flatten(), predict.flatten(), args.num_classes)

            # there is no need to transform the one-hot array to visual RGB array
            # predict = colour_code_segmentation(np.array(predict), label_info)
            # label = colour_code_segmentation(np.array(label), label_info)
            precision_record.append(precision)

            tq.update(1)

        tq.close()
        precision = np.mean(precision_record)
        miou_list = per_class_iu(hist)
        miou = np.mean(miou_list)
        print('\nprecision per pixel for test: %.3f' % precision)
        print('mIoU for validation: %.3f' % miou)
        print(f'mIoU per class: {miou_list}')

        return precision, miou

## **Plots**

### Loss plot



In [ ]:
"""
tr = [x[0] for x in loss_list]
vl = [x[1] for x in loss_list]

print(tr)
print(vl)

plt.figure()
plt.title("Loss comparison")
plt.plot(list(range(1, NUM_EPOCHS + 1, 1)), tr, label = 'training loss')
plt.plot(list(range(1, NUM_EPOCHS + 1, 1)), vl, label = 'validation loss')
plt.xlabel("# epochs")
plt.ylabel("Loss")
plt.legend()
plt.savefig(PLOTS + f'/loss_{OPTIM}_{np.log10(LR):.0f}_{NUM_EPOCHS}_{SCHEDULER}.png')
plt.show()
"""

'\ntr = [x[0] for x in loss_list]\nvl = [x[1] for x in loss_list]\n\nprint(tr)\nprint(vl)\n\nplt.figure()\nplt.title("Loss comparison")\nplt.plot(list(range(1, NUM_EPOCHS + 1, 1)), tr, label = \'training loss\')\nplt.plot(list(range(1, NUM_EPOCHS + 1, 1)), vl, label = \'validation loss\')\nplt.xlabel("# epochs")\nplt.ylabel("Loss")\nplt.legend()\nplt.savefig(PLOTS + f\'/loss_{OPTIM}_{np.log10(LR):.0f}_{NUM_EPOCHS}_{SCHEDULER}.png\')\nplt.show()\n'

### Accuracy (mIOU) plot

In [ ]:
"""
tr = [x[0] for x in acc_list]
vl = [x[1] for x in acc_list]

plt.figure()
plt.title("Accuracy comparison")
plt.plot(list(range(1, NUM_EPOCHS + 1, 1)), tr, label = 'training accuracy')
plt.plot(list(range(1, NUM_EPOCHS + 1, 1)), vl, label = 'validation accuracy')
plt.xlabel("# epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.savefig(PLOTS + f'/accuracy_{OPTIM}_{np.log10(LR):.0f}_{NUM_EPOCHS}_{SCHEDULER}.png')
plt.show()
"""

'\ntr = [x[0] for x in acc_list]\nvl = [x[1] for x in acc_list]\n\nplt.figure()\nplt.title("Accuracy comparison")\nplt.plot(list(range(1, NUM_EPOCHS + 1, 1)), tr, label = \'training accuracy\')\nplt.plot(list(range(1, NUM_EPOCHS + 1, 1)), vl, label = \'validation accuracy\')\nplt.xlabel("# epochs")\nplt.ylabel("Accuracy")\nplt.legend()\nplt.savefig(PLOTS + f\'/accuracy_{OPTIM}_{np.log10(LR):.0f}_{NUM_EPOCHS}_{SCHEDULER}.png\')\nplt.show()\n'

## **Test**

In [ ]:
"""
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
net.train(False) # Set Network to evaluation mode

# comando duale per caricare la rete migliore in net dal file .pth
# per recuparlo --> load_state_dict()
# torch.nn.Module.load_state_dict() -> it loads a model’s parameter dictionary using a deserialized state_dict. (visit https://pytorch.org/tutorials/beginner/saving_loading_models.html)

running_corrects = 0
for images, labels in tqdm(test_dataloader):
  images = images.to(DEVICE)
  labels = labels.to(DEVICE)

  # Forward Pass
  outputs = net(images)

  # Get predictions
  _, preds = torch.max(outputs.data, 1)

  # Update Corrects
  running_corrects += torch.sum(preds == labels.data).data.item()

# Calculate Accuracy
accuracy = running_corrects / float(len(test_dataset))

print('Test Accuracy: {}'.format(accuracy))
"""

"\nnet = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda\nnet.train(False) # Set Network to evaluation mode\n\n# comando duale per caricare la rete migliore in net dal file .pth\n# per recuparlo --> load_state_dict()\n# torch.nn.Module.load_state_dict() -> it loads a model’s parameter dictionary using a deserialized state_dict. (visit https://pytorch.org/tutorials/beginner/saving_loading_models.html)\n\nrunning_corrects = 0\nfor images, labels in tqdm(test_dataloader):\n  images = images.to(DEVICE)\n  labels = labels.to(DEVICE)\n\n  # Forward Pass\n  outputs = net(images)\n\n  # Get predictions\n  _, preds = torch.max(outputs.data, 1)\n\n  # Update Corrects\n  running_corrects += torch.sum(preds == labels.data).data.item()\n\n# Calculate Accuracy\naccuracy = running_corrects / float(len(test_dataset))\n\nprint('Test Accuracy: {}'.format(accuracy))\n"

## **Main**
Prepare arguments, Datasets, Dataloaders, model, training and test

In [ ]:
def main(args, eval_only=False):

    n_classes = args.num_classes
    mode = args.mode
    train_root = args.train_root
    val_root = args.val_root
    target_root = args.train_root
    domain_adapt = args.domain_adaptation

    # defining the training, validation (and target for domain adaptation) datasets and dataloaders
    if train_root == 'GTA5/':
        if train_root != val_root: # if we only use GTA5 for training, use the entire dataset and don't leave a portion for testing
            train_dataset = GTA5(train_root, 'train', args.training_augmentation, 1)
        else:
            train_dataset = GTA5(train_root, 'train', args.training_augmentation)
    else:
        train_dataset = CityScapes(train_root, 'train')

    if val_root == 'GTA5/':
        val_dataset = GTA5(val_root, 'val')
    else:
        val_dataset = CityScapes(val_root, 'val')

    if target_root == 'GTA5/':
        target_dataset = GTA5(target_root, 'train')
    else:
        target_dataset = CityScapes(target_root, 'train')



    dataloader_train = DataLoader(train_dataset,
                      batch_size=args.batch_size,
                      shuffle=True,
                      num_workers=args.num_workers,
                      pin_memory=False,
                      drop_last=True)

    dataloader_val = DataLoader(val_dataset,
                      batch_size=1,
                      shuffle=False,
                      num_workers=args.num_workers,
                      drop_last=False)

    dataloader_target = DataLoader(target_dataset,
                      batch_size=args.batch_size,
                      shuffle=True,
                      num_workers=args.num_workers,
                      pin_memory=False,
                      drop_last=True)

    ## model
    model = BiSeNet(backbone=args.backbone, n_classes=n_classes, pretrain_model=args.pretrain_path, use_conv_last=args.use_conv_last)

    if args.epoch_start_i != 0:
        print(f'loading data from saved model {args.saved_model}')
        model.load_state_dict(torch.load(f'{args.save_model_path}{args.saved_model}'))

    if torch.cuda.is_available() and args.use_gpu:
        model = torch.nn.DataParallel(model).cuda()

    ## optimizer
    # build optimizer
    if args.optimizer == 'rmsprop':
        optimizer = torch.optim.RMSprop(model.parameters(), args.learning_rate)
    elif args.optimizer == 'sgd':
        optimizer = torch.optim.SGD(model.parameters(), args.learning_rate, momentum=0.9, weight_decay=1e-4)
    elif args.optimizer == 'adam':
        optimizer = torch.optim.Adam(model.parameters(), args.learning_rate)
    else:  # rmsprop
        print('not supported optimizer \n')
        return None

    if domain_adapt:
        # init Discriminator
        model_D = FCDiscriminator(args.num_classes)

        model_D = model_D.cuda()

        if args.optimizer_D == 'rmsprop':
            optimizer_D = torch.optim.RMSprop(model_D.parameters(), args.learning_rate_D)
        elif args.optimizer_D == 'sgd':
            optimizer_D = torch.optim.SGD(model_D.parameters(), args.learning_rate_D, momentum=0.9, weight_decay=1e-4)
        elif args.optimizer_D == 'adam':
            optimizer_D = torch.optim.Adam(model_D.parameters(), args.learning_rate_D, betas=(0.9, 0.99))
        else:  # rmsprop
            print('not supported optimizer \n')
            return None

    if not eval_only: #this is for when we only care about evaluating a saved model and not about training
        if domain_adapt:
            ## train loop for domain adaptation
            train_da(args, model, optimizer, model_D, optimizer_D, dataloader_train, dataloader_val, domain_adapt=domain_adapt, dataloader_target=dataloader_target)
        else:
            ## normal train loop
            train(args, model, optimizer, dataloader_train, dataloader_val)
    # final test
    val(args, model, dataloader_val)

### main execution

In [ ]:
class arguments():
    mode = 'train'
    backbone = 'CatmodelSmall'
    pretrain_path = "/content/Drive/MyDrive/Colab Notebooks/checkpoints/STDCNet813M_73.91.tar"
    use_conv_last = False
    num_epochs = 50
    epoch_start_i = 0
    checkpoint_step = 1
    validation_step = 5
    crop_height = 512
    crop_width = 1024
    batch_size = 8
    learning_rate = 1e-3
    learning_rate_D = 1e-4
    num_workers = 2
    num_classes = 19
    cuda = '0'
    use_gpu = True
    save_model_path = '/content/Drive/MyDrive/Colab Notebooks/Partial models/'
    saved_model = f'DA_Saved_model_epoch_{epoch_start_i}.pth'
    optimizer = 'adam'
    optimizer_D = 'adam'
    loss = 'crossentropy'
    #'Cityscapes/Cityspaces/'
    #'GTA5/'
    train_root = 'GTA5/'
    val_root ='Cityscapes/Cityspaces/'
    target_root = 'Cityscapes/Cityspaces/'
    training_augmentation = True
    domain_adaptation = True
main_args = arguments()

main(main_args, eval_only=False)

use pretrain model /content/Drive/MyDrive/Colab Notebooks/checkpoints/STDCNet813M_73.91.tar
start train with domain adaptation
Train DA





  0%|          | 0/1664 [00:00<?, ?it/s]


epoch 1, lr 0.005000, lr_discriminator 0.001000:   0%|          | 0/1664 [00:00<?, ?it/s]<ipython-input-21-5a81cf3a6fe3>:87: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  D_out = model_D(F.softmax(output_t))
<ipython-input-21-5a81cf3a6fe3>:105: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  D_out = model_D(F.softmax(output_s))                                                                   # we feed the discriminator with the output of the G-model
<ipython-input-21-5a81cf3a6fe3>:115: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  D_out = model_D(F.softmax(output_t))  # we feed the discriminator with the output of the model



epoch 1, lr 0.005000, lr_discriminator 0.001000:   0%|          | 4/1664 [00:02<15:46,  1

loss for train :
 - Segmentation: 2.555654521458424
 - Adversarial: 0.0006967954606387558
 - Discriminator: 0.34569264776431596





  0%|          | 0/1664 [00:00<?, ?it/s]


epoch 2, lr 0.004910, lr_discriminator 0.000982:   0%|          | 0/1664 [00:00<?, ?it/s]


epoch 2, lr 0.004910, lr_discriminator 0.000982:   0%|          | 4/1664 [00:02<20:19,  1.36it/s]


epoch 2, lr 0.004910, lr_discriminator 0.000982:   0%|          | 4/1664 [00:02<20:19,  1.36it/s, loss_seg=1.784923, loss_adv=0.000697, loss_D=0.344874]


epoch 2, lr 0.004910, lr_discriminator 0.000982:   0%|          | 8/1664 [00:03<10:59,  2.51it/s, loss_seg=1.784923, loss_adv=0.000697, loss_D=0.344874]


epoch 2, lr 0.004910, lr_discriminator 0.000982:   0%|          | 8/1664 [00:03<10:59,  2.51it/s, loss_seg=2.966754, loss_adv=0.000709, loss_D=0.339106]


epoch 2, lr 0.004910, lr_discriminator 0.000982:   1%|          | 12/1664 [00:04<08:26,  3.26it/s, loss_seg=2.966754, loss_adv=0.000709, loss_D=0.339106]


epoch 2, lr 0.004910, lr_discriminator 0.000982:   1%|          | 12/1664 [00:04<08:26,  3.26it/s, loss_seg=1.932912, loss_adv=0.000714, loss

loss for train :
 - Segmentation: 1.8362338825200613
 - Adversarial: 0.0006966333348827902
 - Discriminator: 0.3471884550526738





  0%|          | 0/1664 [00:00<?, ?it/s]


epoch 3, lr 0.004820, lr_discriminator 0.000964:   0%|          | 0/1664 [00:00<?, ?it/s]


epoch 3, lr 0.004820, lr_discriminator 0.000964:   0%|          | 4/1664 [00:03<24:21,  1.14it/s]


epoch 3, lr 0.004820, lr_discriminator 0.000964:   0%|          | 4/1664 [00:03<24:21,  1.14it/s, loss_seg=1.942274, loss_adv=0.000675, loss_D=0.363141]


epoch 3, lr 0.004820, lr_discriminator 0.000964:   0%|          | 8/1664 [00:04<13:11,  2.09it/s, loss_seg=1.942274, loss_adv=0.000675, loss_D=0.363141]


epoch 3, lr 0.004820, lr_discriminator 0.000964:   0%|          | 8/1664 [00:04<13:11,  2.09it/s, loss_seg=1.661440, loss_adv=0.000687, loss_D=0.350232]


epoch 3, lr 0.004820, lr_discriminator 0.000964:   1%|          | 12/1664 [00:05<10:24,  2.65it/s, loss_seg=1.661440, loss_adv=0.000687, loss_D=0.350232]


epoch 3, lr 0.004820, lr_discriminator 0.000964:   1%|          | 12/1664 [00:05<10:24,  2.65it/s, loss_seg=1.193208, loss_adv=0.000765, loss

loss for train :
 - Segmentation: 1.7098187569242258
 - Adversarial: 0.0006953962807668946
 - Discriminator: 0.34592852204178387





  0%|          | 0/1664 [00:00<?, ?it/s]


epoch 4, lr 0.004729, lr_discriminator 0.000946:   0%|          | 0/1664 [00:00<?, ?it/s]


epoch 4, lr 0.004729, lr_discriminator 0.000946:   0%|          | 4/1664 [00:02<16:51,  1.64it/s]


epoch 4, lr 0.004729, lr_discriminator 0.000946:   0%|          | 4/1664 [00:02<16:51,  1.64it/s, loss_seg=4.826578, loss_adv=0.000687, loss_D=0.349461]


epoch 4, lr 0.004729, lr_discriminator 0.000946:   0%|          | 8/1664 [00:03<09:39,  2.86it/s, loss_seg=4.826578, loss_adv=0.000687, loss_D=0.349461]


epoch 4, lr 0.004729, lr_discriminator 0.000946:   0%|          | 8/1664 [00:03<09:39,  2.86it/s, loss_seg=1.730145, loss_adv=0.000682, loss_D=0.352176]


epoch 4, lr 0.004729, lr_discriminator 0.000946:   1%|          | 12/1664 [00:03<07:57,  3.46it/s, loss_seg=1.730145, loss_adv=0.000682, loss_D=0.352176]


epoch 4, lr 0.004729, lr_discriminator 0.000946:   1%|          | 12/1664 [00:03<07:57,  3.46it/s, loss_seg=1.683160, loss_adv=0.000686, loss